# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-24-events.c

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

In [3]:
import create_datafile_new
import create_event_data
import do_create_tables
import etl
import verify
import db_executor
import query_cql
import utils

## Read data from /event_data/, and transform them as csv file

In [4]:
create_datafile_new.main()
#create_event_data.main()

/home/workspace
['/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-24-events.c

## Creating a Cluster and Keyspace
## Creating tables for querying later

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION =
    {'class' : 'SimpleStrategy','replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

In [7]:
do_create_tables.main()

Establishing connection...
Dropping old tables
Creating new tables
Preparing done!..Closing connection...
All Done.


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Processing data by etl.py

In [8]:
etl.main()

Creating connection...
Inserting data...
Shut down connection...
Done.


### Verify the result              

In [9]:
# Add in the SELECT statement to verify the data was entered into the table
# query1 = "SELECT * FROM music_library WHERE sessionId='338' AND itemInSession='4'"
session.set_keyspace('sparkifydb')
query1 = "SELECT count(1) FROM song_by_user"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)    
for row in rows:
    print (row)

Row(count=7770)


### Check the result by executing 3 cql statements

In [10]:
verify.main()

Get connection...
Verifying.....
--------------------------------

SELECT artist, song, length
FROM songs_by_session
WHERE
    sessionId = 338
AND
    itemInSession = 4

      artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312
--------------------------------

SELECT artist, song, firstName, lastName
FROM song_by_user
WHERE
    userId=10
AND
    sessionId = 182


              artist                                                  song firstname lastname
0   Down To The Bone                                    Keep On Keepin' On    Sylvie     Cruz
1       Three Drives                                           Greece 2000    Sylvie     Cruz
2  Sebastien Tellier                                             Kilometer    Sylvie     Cruz
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio Edit)    Sylvie     Cruz
--------------------------------

SELECT firstName, lastName
FROM user_by_song
WHERE
    song = 'All Hand

In [12]:
session.shutdown()
cluster.shutdown()